
# Arabic RAG Chatbot Project

#### Build a simple Arabic QA chatbot using Retrieval-Augmented Generation (RAG)


## Load the Dataset
Using the ARCD dataset from Hugging Face — it has about 1.4k Arabic QA pairs


In [1]:
from datasets import load_dataset
import pandas as pd

# Load data
dataset = load_dataset("hsseinmz/arcd")
df_train = pd.DataFrame(dataset["train"])
df_val = pd.DataFrame(dataset["validation"])


In [6]:
# Show sample
df_train.head()

,id,title,context,question,answers
0,969331847966,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- من هو جمال أحمد حمزة خاشقجي؟,"{'text': ['صحفي وإعلامي'], 'answer_start': [73]}"
1,115150665555,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- متى ولد جمال أحمد حمزة خاشقجي وتوفي؟ ال,{'text': ['حمزة خاشقجي (13 أكتوبر 1958، المدين...
2,74212080718,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- في أي مدينة ولد جمال أحمد حمزة خاشقجي؟ ال,"{'text': ['المدينة المنورة'], 'answer_start': ..."
3,465699296586,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- في أي صحيفة قام بكتابة عمود منذ عام 2017؟ ال,"{'text': ['واشنطن بوست'], 'answer_start': [224]}"
4,564177542570,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- كيف وصفها في الصحف ووسائل الإعلام الدولية؟ ال,{'text': ['وُصف في الصحف وأجهزة الاعلام العالم...


In [7]:
print(df_train.iloc[0])


id                                               969331847966
title                                             جمال خاشقجي
context     جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...
question                       - من هو جمال أحمد حمزة خاشقجي؟
answers      {'text': ['صحفي وإعلامي'], 'answer_start': [73]}
Name: 0, dtype: object


In [8]:
# Just checking sizes and missing values
print(f"Training Data Size: {len(df_train)}")
print(f"Validation Data Size: {len(df_val)}")
print("\nMissing values in training data:")
print(df_train.isnull().sum())

Training Data Size: 693
Validation Data Size: 702

Missing values in training data:
id          0
title       0
context     0
question    0
answers     0
dtype: int64


## Text Cleaning & Normalization
Arabic text can be messy sometimes (extra marks, diacritics, etc.), so here I just clean and normalize it by removing diacritics, unwanted characters, and extra spaces to keep everything consistent.


In [9]:
import re

def normalize_arabic(text):
    # Normalize Arabic text
    text = re.sub(r'[\u0617-\u061A\u064B-\u0652]', '', text)
    text = re.sub(r'[^\u0600-\u06FF0-9\s؟.,:؛!]', '', text)
    return re.sub(r'\s+', ' ', text).strip()

df_train_raw = df_train.copy()
# clean columns
for df in [df_train, df_val]:
    for col in ['context', 'question']:
        df[col] = df[col].astype(str).apply(normalize_arabic)
    df['answer_text'] = df['answers'].apply(lambda x: x['text'][0] if x['text'] else "")
    df['answer_text'] = df['answer_text'].astype(str).apply(normalize_arabic)


In [10]:
# Just showing before and after cleaning
sample_text = df_train_raw.loc[20, "context"]
print(" Before cleaning:\n", sample_text)
print("\n After cleaning:\n", normalize_arabic(sample_text))

 Before cleaning:
 مِصرَ أو (رسمياً: جُمهورِيّةُ مِصرَ العَرَبيّةِ) هي دولة عربية تقع في الركن الشمالي الشرقي من قارة أفريقيا، ولديها امتداد آسيوي، حيث تقع شبه جزيرة سيناء داخل قارة آسيا فهي دولة عابرة للقارات، قُدّر عدد سكانها بـ104 مليون نسمة، ليكون ترتيبها الثالثة عشر بين دول العالم بعدد السكان والأكثر سكانا عربيا.

 After cleaning:
 مصر أو رسميا: جمهورية مصر العربية هي دولة عربية تقع في الركن الشمالي الشرقي من قارة أفريقيا، ولديها امتداد آسيوي، حيث تقع شبه جزيرة سيناء داخل قارة آسيا فهي دولة عابرة للقارات، قدر عدد سكانها بـ104 مليون نسمة، ليكون ترتيبها الثالثة عشر بين دول العالم بعدد السكان والأكثر سكانا عربيا.


## Generate Embeddings
Generated embeddings for all the contexts using an Arabic transformer model


In [2]:
from sentence_transformers import SentenceTransformer

# Load Arabic embeddings model
embeddings_model = SentenceTransformer('Omartificial-Intelligence-Space/Arabic-Triplet-Matryoshka-V2')

def generate_embeddings(texts):
    """encode texts and normalize thems """
    return embeddings_model.encode(
        texts,
        batch_size=16,
        normalize_embeddings=True,
        convert_to_tensor=True
    )

train_context_emb = generate_embeddings(df_train["context"].tolist())
val_context_emb = generate_embeddings(df_val["context"].tolist())
print(f"Training Contexts Embeddings Shape: {train_context_emb.shape}")


c:\Users\areen\anaconda3\envs\ml_env\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)



Training Contexts Embeddings Shape: torch.Size([693, 768])


## Qdrant Setup (Vector Database)
Here I connected to Qdrant, a vector database, to store all the context embeddings for easy retrieval.


In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct

# connect to Qdrant
client = QdrantClient("http://localhost:6333")


collection_name = "arabic_qa"

# recreate the collection
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=train_context_emb.shape[1],
        distance=Distance.COSINE
    )
)
# Upload TRAIN points
train_points = [
    PointStruct(
        id=i,
        vector=train_context_emb[i].cpu().numpy().tolist(),
        payload={
            "context": df_train.iloc[i]["context"],
            "question": df_train.iloc[i]["question"],
            "answer_text": df_train.iloc[i]["answer_text"],
            "id": df_train.iloc[i]["id"],
            "title": df_train.iloc[i]["title"],
            "split": "train"
        }
    )
    for i in range(len(df_train))
]

# Upload VALIDATION points
val_points = [
    PointStruct(
        id=len(df_train) + i,
        vector=val_context_emb[i].cpu().numpy().tolist(),
        payload={
            "context": df_val.iloc[i]["context"],
            "question": df_val.iloc[i]["question"],
            "split": "val"
        }
    )
    for i in range(len(df_val))
]

all_points = train_points + val_points

operation_info = client.upsert(
    collection_name=collection_name,
    wait=True,
    points=all_points
)


/tmp/ipython-input-2087995664.py:10: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


In [13]:
print("Upsert complete:", operation_info.status)
print("Total vectors stored:", len(all_points))

Upsert complete: completed
Total vectors stored: 1395


##  Retrieval Function
This part finds the most similar contexts from Qdrant based on the question the user asks.

In [14]:

def retrieve_similar_context(query, top_k=5):
    query_embedding = embeddings_model.encode(
        query,
        normalize_embeddings=True,
        convert_to_tensor=False
    ).astype("float32")

    search_results = client.query_points(
        collection_name=collection_name,
        query=query_embedding,
        limit=top_k * 3
    )

    unique_contexts = {}
    for result in search_results.points:
        context = result.payload["context"]
        answer = result.payload.get("answer_text", "")
        score = result.score

        # If context exists, keep only the highest score
        if context not in unique_contexts or score > unique_contexts[context]["score"]:
            unique_contexts[context] = {"context": context, "answer": answer, "score": score}

    # Return top_k distinct contexts sorted by score
    sorted_results = sorted(unique_contexts.values(), key=lambda x: x["score"], reverse=True)
    return sorted_results[:top_k]



In [15]:
# quick test
user_question = "متى ولد جمال خاشقجي؟"
similar_contexts = retrieve_similar_context(user_question)

print(" Top-k Retrieved Contexts:")
for i, context in enumerate(similar_contexts, start=1):
    print(f"{i}. [Score: {context['score']:.4f}]")
    print(f"   Context: {context['context']}")


 Top-k Retrieved Contexts:
1. [Score: 0.7705]
   Context: جمال أحمد حمزة خاشقجي 13 أكتوبر 1958، المدينة المنورة 2 أكتوبر 2018، صحفي وإعلامي سعودي، رأس عدة مناصب لعدد من الصحف في السعودية، وتقلد منصب مستشار، كما أنه مدير عام قناة العرب الإخبارية سابقا.
2. [Score: 0.7653]
   Context: جمال أحمد حمزة خاشقجي 13 أكتوبر 1958، المدينة المنورة 2 أكتوبر 2018، صحفي وإعلامي سعودي، رأس عدة مناصب لعدد من الصحف في السعودية، وتقلد منصب مستشار، كما أنه مدير عام قناة العرب الإخبارية سابقا. ويكتب عمودا في صحيفة واشنطن بوست منذ 2017، وصف في الصحف وأجهزة الاعلام العالمية بأنه وفي للدولة السعودية ومنتقد لسياساتها.
3. [Score: 0.6499]
   Context: غادر خاشقجي السعودية في سبتمبر 2017، وكتب بعد ذلك مقالات صحفية انتقد فيها الحكومة السعودية. انتقد خاشقجي بصورة كبيرة ولي العهد السعودي محمد بن سلمان، والملك سلمان بن عبد العزيز. وكذلك عارض التدخل العسكري في اليمن.
4. [Score: 0.4223]
   Context: ترامب هو الابن الرابع لعائلة مكونة من خمسة أطفال، والده فريد ترامب أحد الأثرياء وملاك العقارات في مدينة نيويورك، وقد تأثر دو

In [16]:
# quick test
user_question = "في أي علم ولد ألبرت أينشتاين؟"
similar_contexts = retrieve_similar_context(user_question)

print(" Top-k Retrieved Contexts:")
for i, context in enumerate(similar_contexts, start=1):
    print(f"{i}. [Score: {context['score']:.4f}]")
    print(f"   Context: {context['context']}")


 Top-k Retrieved Contexts:
1. [Score: 0.8055]
   Context: ألبرت أينشتاين بالألمانية: 14 مارس 1879 18 أبريل 1955 عالم فيزياء ألماني المولد، حيث تخلى عن الجنسية الألمانية لاحقا سويسري وأمريكي الجنسية، من أبوين يهوديين، وهو يشتهر بأب النسبية كونه واضع النسبية الخاصة والنسبية العامة الشهيرتين اللتين كانتا اللبنة الأولى للفيزياء النظرية الحديثة، ولقد حاز في عام 1921 على جائزة نوبل في الفيزياء عن ورقة بحثية عن التأثير الكهروضوئي، ضمن ثلاثمائة ورقة علمية أخرى له في تكافؤ المادة والطاقة وميكانيكا الكم وغيرها، وأدت استنتاجاته المبرهنة إلى تفسير العديد من الظواهر العلمية التي فشلت الفيزياء الكلاسيكية في إثباتها.
2. [Score: 0.8025]
   Context: عاش أينشتاين في سويسرا بين عامي 1895 1914، باستثناء عام واحد في براغ، وحصل على دبلومه الأكاديمي من المدرسة التقنية الفدرالية السويسرية في زيورخ في عام 1900. حصل على الجنسية السويسرية في عام 1901، احتفظ بها لبقية حياته بعد أن أصبح بلا جنسية لأكثر من خمس سنوات. في عام 1905، حصل على درجة الدكتوراه من جامعة زيورخ. في العام نفسه، نشر أربع ورقات رائدة سميت تلك ال

## RAG Evaluation

In [17]:
from sentence_transformers import util
import math
from tqdm import tqdm

top_k = 3
similarity_threshold = 0.60

retrieval_precisions = []
retrieval_recalls = []
retrieval_f1s = []
average_precisions = []
reciprocal_ranks = []
ndcgs = []

soft_hits = 0
total_sbert_sim = 0
total = 0

# NDCG
def compute_ndcg(relevance, k):
    dcg = 0.0
    for i in range(min(k, len(relevance))):
        if relevance[i] == 1:
            dcg += 1 / math.log2(i + 2)

    ideal = sorted(relevance, reverse=True)
    idcg = 0.0
    for i in range(min(k, len(ideal))):
        if ideal[i] == 1:
            idcg += 1 / math.log2(i + 2)

    return dcg / idcg if idcg > 0 else 0.0


# Check if paragraph contains answer semantically
def contains_answer_semantically(gold_answer, retrieved_paragraphs, threshold=0.70):
    if not gold_answer.strip():
        return False

    ans_emb = embeddings_model.encode(gold_answer, convert_to_tensor=True)
    paras_emb = embeddings_model.encode(
        retrieved_paragraphs,  # list of paragraph texts
        convert_to_tensor=True
    )
    sims = util.cos_sim(ans_emb, paras_emb)[0]
    return any(sim.item() >= threshold for sim in sims)


# MAIN Evaluation Loop

for i, row in tqdm(df_val.iterrows(), total=len(df_val)):

    question = row["question"]
    gold_answer = row["answer_text"]
    gold_context = row["context"]  # this is one paragraph

    # Retrieve top-k paragraphs
    retrieved = retrieve_similar_context(question, top_k=top_k)
    retrieved_contexts = [r["context"] for r in retrieved]  # list of paragraphs

    # SBERT similarity between gold paragraph and retrieved paragraphs
    gold_emb = embeddings_model.encode(gold_context, convert_to_tensor=True)
    sims = []

    for para in retrieved_contexts:
        para_emb = embeddings_model.encode(para, convert_to_tensor=True)
        sims.append(util.cos_sim(gold_emb, para_emb).item())

    max_sim = max(sims)
    total_sbert_sim += max_sim
    total += 1

    if max_sim >= similarity_threshold:
        soft_hits += 1

    # Convert sims → relevance
    relevance = [1 if s >= similarity_threshold else 0 for s in sims]

    # Precision / Recall / F1
    precision = sum(relevance) / top_k
    recall = 1 if any(relevance) else 0
    f1 = (2 * precision * recall) / (precision + recall + 1e-8)

    retrieval_precisions.append(precision)
    retrieval_recalls.append(recall)
    retrieval_f1s.append(f1)

    # Average Precision (AP)
    ap = sum([(sum(relevance[:i+1]) / (i+1)) * r for i, r in enumerate(relevance)])
    ap = ap / max(1, sum(relevance))
    average_precisions.append(ap)

    # MRR
    if 1 in relevance:
        rr = 1 / (relevance.index(1) + 1)
    else:
        rr = 0
    reciprocal_ranks.append(rr)

    # NDCG
    ndcgs.append(compute_ndcg(relevance, top_k))


100%|██████████| 702/702 [02:35<00:00,  4.52it/s]


In [19]:
# Final Evaluation Results

mean_precision = sum(retrieval_precisions) / len(retrieval_precisions)
mean_recall    = sum(retrieval_recalls) / len(retrieval_recalls)
mean_f1        = sum(retrieval_f1s) / len(retrieval_f1s)
mean_ap        = sum(average_precisions) / len(average_precisions)
mean_rr        = sum(reciprocal_ranks) / len(reciprocal_ranks)
mean_ndcg      = sum(ndcgs) / len(ndcgs)

print("RAG Retrieval Metrics\n")

print(f"Num Validation Samples          : {total}")
print(f"SoftRecall@{top_k}              : {soft_hits / total:.4f}")
print(f"Avg SBERT Similarity            : {total_sbert_sim / total:.4f}")
print("--------------------------------------")
print(f"Precision@{top_k}               : {mean_precision:.4f}")
print(f"Recall@{top_k}                  : {mean_recall:.4f}")
print(f"F1@{top_k}                      : {mean_f1:.4f}")
print(f"MAP                             : {mean_ap:.4f}")
print(f"MRR                             : {mean_rr:.4f}")
print(f"NDCG@{top_k}                    : {mean_ndcg:.4f}")


RAG Retrieval Metrics

Num Validation Samples          : 702
SoftRecall@3              : 0.8675
Avg SBERT Similarity            : 0.9062
--------------------------------------
Precision@3               : 0.6429
Recall@3                  : 0.8675
F1@3                      : 0.7124
MAP                             : 0.8243
MRR                             : 0.8307
NDCG@3                    : 0.8371


## Generation


In [ ]:
def generate_prompt(question, retrieved_contexts):
    prompt = "أجب على السؤال التالي بناءً على المعلومات المتاحة أدناه:\n\n"

    for i, ctx in enumerate(retrieved_contexts, 1):
        prompt += f"المعلومة {i}: {ctx['context']}\n"

    prompt += f"\nالسؤال: {question}\nالإجابة :"
    return prompt

In [ ]:
# Simple function that cuts the answer
def truncate_answer(answer, stop_chars=['.', '،']):
    for char in stop_chars:
        if char in answer:
            return answer.split(char)[0] + char
    return answer

### Arabic GPT-2
Used an Arabic GPT-2 model to generate answers , basically combining the question with the retrieved context to make a full response.

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load Arabic GPT-2
gpt2_tokenizer = AutoTokenizer.from_pretrained("aubmindlab/aragpt2-medium")
gpt2_model = AutoModelForCausalLM.from_pretrained("aubmindlab/aragpt2-medium")


In [23]:
def generate_answer_gpt2(question, top_k=1, max_new_tokens=30):

    retrieved_contexts = retrieve_similar_context(question, top_k=top_k)
    prompt_text = generate_prompt(question, retrieved_contexts)
    inputs = gpt2_tokenizer(prompt_text, return_tensors="pt", truncation=True, max_length=512)

    # generate the text
    output = gpt2_model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        no_repeat_ngram_size=3,
        do_sample=False ,
        top_p=0.8,
        num_beams=8,
        temperature=0.7,
        pad_token_id=gpt2_tokenizer.eos_token_id,
        eos_token_id=gpt2_tokenizer.eos_token_id

    )

    # decode answer
    answer = gpt2_tokenizer.decode(output[0], skip_special_tokens=True)
    final_answer = answer.replace(prompt_text, "").strip()
    final_answer = truncate_answer(final_answer)


    return final_answer


In [24]:
def generate_answer_gpt2_no_rag(question, max_new_tokens=30):
    inputs = gpt2_tokenizer(question, return_tensors="pt", truncation=True, max_length=512)

    output = gpt2_model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        no_repeat_ngram_size=3,
        do_sample=False,
        top_p=0.8,
        num_beams=8,
        temperature=0.7,
        pad_token_id=gpt2_tokenizer.eos_token_id,
        eos_token_id=gpt2_tokenizer.eos_token_id
    )

    answer = gpt2_tokenizer.decode(output[0], skip_special_tokens=True)
    final_answer = answer.replace(question, "").strip()
    return final_answer



### Gemini

In [ ]:
import google.generativeai as genai

GOOGLE_API_KEY = "GOOGLE_API_KEY"

genai.configure(api_key=GOOGLE_API_KEY)

# Load Gemini Flash model
gemini_model = genai.GenerativeModel("gemini-2.5-flash")


In [26]:
def generate_answer_gemini(question, top_k=1, max_output_tokens=1024, temperature=0.1, top_p=0.9):
    try:
        # Retrieve contexts and build prompt
        retrieved_contexts = retrieve_similar_context(question, top_k=top_k)
        prompt_text = generate_prompt(question, retrieved_contexts)

        # Generate response
        response = gemini_model.generate_content(
            prompt_text,
            generation_config={
                "temperature": temperature,
                "top_p": top_p,
                "max_output_tokens": max_output_tokens,
            }
        )

        # Extract answer
        candidate = response.candidates[0] if hasattr(response, 'candidates') and response.candidates else None
        if candidate and hasattr(candidate, 'content') and candidate.content.parts:
            answer = "".join([p.text for p in candidate.content.parts]).strip()
            return truncate_answer(answer)


        return "No response generated"

    except Exception as e:
        return f"Error: {str(e)}"


In [27]:
def generate_answer_gemini_no_rag(question, top_k=1, max_output_tokens=1024, temperature=0.1, top_p=0.9):
    try:
        response = gemini_model.generate_content(
            question,
            generation_config={
                "temperature": temperature,
                "top_p": top_p,
                "max_output_tokens": max_output_tokens,
            }
        )

        candidate = response.candidates[0] if hasattr(response, 'candidates') and response.candidates else None
        if candidate and candidate.content.parts:
            answer = "".join([p.text for p in candidate.content.parts]).strip()
            return truncate_answer(answer)

        return "No response"

    except Exception as e:
        return f"Error: {str(e)}"


In [37]:
question = "في أي عام ولد ألبرت أينشتاين؟"

gpt2_ans_withRAG = generate_answer_gpt2(question)
gemini_ans_withRAG = generate_answer_gemini(question)
gpt2_ans_withoutRAG = generate_answer_gpt2_no_rag(question)
gemini_ans_withoutRAG = generate_answer_gemini_no_rag(question)

print("Testing GPT-2 & Gemini Flash (RAG vs No-RAG)\n")
print("question")
print(question)
print("\n==============================\n")

print("1) GPT-2 with RAG:")
print(gpt2_ans_withRAG)
print("\n------------------------------\n")

print("2) Gemini Flash with RAG:")
print(gemini_ans_withRAG)
print("\n------------------------------\n")

print("3) GPT-2 without RAG:")
print(gpt2_ans_withoutRAG)
print("\n------------------------------\n")

print("4) Gemini Flash without RAG:")
print(gemini_ans_withoutRAG)


Testing GPT-2 & Gemini Flash (RAG vs No-RAG)

question
في أي عام ولد ألبرت أينشتاين؟


1) GPT-2 with RAG:
ولد ألبرت إينشتاين في عام 1879 م في مدينة لايبزغ في ألمانيا ،

------------------------------

2) Gemini Flash with RAG:
في عام 1879.

------------------------------

3) GPT-2 without RAG:
؟ ؟ ؟ ! ! ! . . . في عام 1901 م ، ولد ألبرت إينشتاين ، وهو عالم فيزيائي ألماني ، في مدينة

------------------------------

4) Gemini Flash without RAG:
ولد ألبرت أينشتاين في عام **1879**.


In [36]:
question = "ما عاصمة فلسطين؟ "

gpt2_ans_withRAG = generate_answer_gpt2(question)
gemini_ans_withRAG = generate_answer_gemini(question)
gpt2_ans_withoutRAG = generate_answer_gpt2_no_rag(question)
gemini_ans_withoutRAG = generate_answer_gemini_no_rag(question)

print("Testing GPT-2 & Gemini Flash (RAG vs No-RAG)\n")
print("question")
print(question)
print("\n==============================\n")

print("1) GPT-2 with RAG:")
print(gpt2_ans_withRAG)
print("\n------------------------------\n")

print("2) Gemini Flash with RAG:")
print(gemini_ans_withRAG)
print("\n------------------------------\n")

print("3) GPT-2 without RAG:")
print(gpt2_ans_withoutRAG)
print("\n------------------------------\n")

print("4) Gemini Flash without RAG:")
print(gemini_ans_withoutRAG)


Testing GPT-2 & Gemini Flash (RAG vs No-RAG)

question
ما عاصمة فلسطين؟ 


1) GPT-2 with RAG:
عاصمة فلسطين هي مدينة القدس ،

------------------------------

2) Gemini Flash with RAG:
بناءً على المعلومات المتاحة أعلاه، لم يتم ذكر عاصمة فلسطين بشكل صريح.

------------------------------

3) GPT-2 without RAG:
م . ل . ؟ ؟ ؟ - ي . ع . - ( أ . ف . ب )

------------------------------

4) Gemini Flash without RAG:
العاصمة المعلنة لدولة فلسطين هي **القدس**، وبشكل خاص **القدس الشرقية**، التي يعتبرها الفلسطينيون عاصمتهم المستقبلية.


## Evaluation
calculated BLEU-2, BLEU-4, EM, SIM, F1 just to check how good the model’s answers are.

In [4]:
import time
import numpy as np
import random
import torch
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize
from sentence_transformers import util
from evaluate import load
import nltk
nltk.download('punkt_tab')

smooth = SmoothingFunction().method1


# LOAD METRICS
bleu_metric = load("bleu")


# NORMALIZATION + EXACT MATCH
def exact_match(pred, true):
    p = pred.strip().lower()
    t = true.strip().lower()

    import re
    p = re.sub(r'[^\w\s]', '', p)
    t = re.sub(r'[^\w\s]', '', t)

    if p == t: return 1.0
    if p in t: return 1.0
    if t in p: return 1.0
    return 0.0


# BINARY TOKEN OVERLAP (F1)
def binary_token_f1(preds, refs):
    total_tp = total_fp = total_fn = 0

    for pred, gold in zip(preds, refs):
        pred_tokens = set(word_tokenize(pred.lower()))
        gold_tokens = set(word_tokenize(gold.lower()))

        tp = len(pred_tokens & gold_tokens)
        fp = len(pred_tokens - gold_tokens)
        fn = len(gold_tokens - pred_tokens)

        total_tp += tp
        total_fp += fp
        total_fn += fn

    precision = total_tp / (total_tp + total_fp + 1e-8)
    recall    = total_tp / (total_tp + total_fn + 1e-8)
    f1        = 2 * precision * recall / (precision + recall + 1e-8)

    return f1

# COMPUTE ALL SCORES (BLEU-2, BLEU-4, EM, SIM, F1)
def compute_scores(predictions, references):
    results = {}

    # BLEU-2
    results["bleu2"] = np.mean([
        sentence_bleu([r.split()], p.split(),
                      smoothing_function=smooth,
                      weights=(0.5, 0.5))
        for p, r in zip(predictions, references)
    ])

    # BLEU-4 (using HF evaluate)
    bleu4 = bleu_metric.compute(
        predictions=predictions,
        references=[[r] for r in references],
        max_order=4
    )["bleu"]
    results["bleu4"] = bleu4

    # F1 token-based
    results["f1"] = binary_token_f1(predictions, references)


    # Exact match
    results["em"] = np.mean([exact_match(p, r)
                             for p, r in zip(predictions, references)])

    # Semantic similarity
    emb_pred = embeddings_model.encode(predictions, normalize_embeddings=True, convert_to_tensor=True)
    emb_true = embeddings_model.encode(references, normalize_embeddings=True, convert_to_tensor=True)
    results["sim"] = util.cos_sim(emb_pred, emb_true).diagonal().cpu().numpy().mean()

    return results


# GEMINI SAFE GENERATION
def safe_gemini_generate(question, use_rag=True, max_retries=5, base_delay=10):
    for attempt in range(max_retries):
        try:
            if use_rag:
                ans = generate_answer_gemini(question)
            else:
                ans = generate_answer_gemini_no_rag(question)

            return normalize_arabic(ans)

        except Exception as e:
            if "503" in str(e) or "429" in str(e) or "quota" in str(e).lower():
                delay = base_delay * (2 ** attempt) + random.uniform(0, 2)
                print(f"Gemini rate limit (attempt {attempt+1}). Sleeping {delay:.1f}s ...")
                time.sleep(delay)
            else:
                print("Unexpected error:", e)
                return "ERROR"

    return "ERROR"



[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\areen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [31]:

df_subset = df_val.head(50)

pred_gpt2 = []
pred_gpt2_no_rag = []
pred_gemini = []
pred_gemini_no_rag = []
refs = []

print("Starting full 4-model evaluation...\n")

for i, row in df_subset.iterrows():
    question = row["question"]
    true_answer = normalize_arabic(row["answer_text"])
    refs.append(true_answer)

    print(f"Processing {i+1}/50: {question[:40]}...")

    # GPT-2 with RAG
    try:
        pred_gpt2.append(normalize_arabic(generate_answer_gpt2(question)))
    except:
        pred_gpt2.append("ERROR")

    # GPT-2 without RAG
    try:
        pred_gpt2_no_rag.append(normalize_arabic(generate_answer_gpt2_no_rag(question)))
    except:
        pred_gpt2_no_rag.append("ERROR")

    # GEMINI with RAG
    pred_gemini.append(safe_gemini_generate(question, use_rag=True))

    # GEMINI without RAG
    pred_gemini_no_rag.append(safe_gemini_generate(question, use_rag=False))

    time.sleep(5)


# COMPUTE SCORES
gpt2_results = compute_scores(pred_gpt2, refs)
gpt2_no_rag_results = compute_scores(pred_gpt2_no_rag, refs)
gemini_results = compute_scores(pred_gemini, refs)
gemini_no_rag_results = compute_scores(pred_gemini_no_rag, refs)


Starting full 4-model evaluation...

Processing 1/50: من هو حمزة بن عبد المطلب؟...
Processing 2/50: بما وصفه رسول الله؟...
Processing 3/50: بما وصف رسول الله على ؟...
Processing 4/50: متى اسلم حمزة؟...
Processing 5/50: و ماذا فعل فى غزوة بدر؟...
Processing 6/50: من قتل حمزة؟...
Processing 7/50: من هو عبد المطلب بن هاشم؟...
Processing 8/50: من اخو حمزة فى الرضاعة؟...
Processing 9/50: ماذا جمع بين حمزة و محمد بن عبد الله؟...
Processing 10/50: ما هو القمر؟...
Processing 11/50: كم يبلغ قطره؟...
Processing 12/50: م تبلغ كتلته؟...
Processing 13/50: ما هى سمات القمر؟...
Processing 14/50: كم المسافة المدارية للقمر؟...
Processing 15/50: ما اللذى يتسبب فى ظهور القمر بنفس حجمه ت...
Processing 16/50: لمن اول سفينة فضائية وصلت للقمر؟...
Processing 17/50: متى وصلت اول بعثة بشرية للقمر؟...
Processing 18/50: الى متى يرجع اصل نشأة و تكوين القمر؟...
Processing 19/50: اين تقع مكة المكرمة؟...
Processing 20/50: كم تبعد مكة المكرمة عن المدينةالمنورة؟...
Processing 21/50: كم تبعد مكة المكرمة عن الطائف؟...
Pr

In [39]:

def print_metrics(name, r):
    print(f"\n===== {name} =====")
    print(f"BLEU-2:              {r['bleu2']:.4f}")
    print(f"F1:                  {r['f1']:.4f}")
    print(f"Exact Match:         {r['em']:.4f}")
    print(f"Semantic Similarity: {r['sim']:.4f}")


print_metrics("GPT-2 (with RAG)", gpt2_results)
print_metrics("GPT-2 (without RAG)", gpt2_no_rag_results)
print_metrics("Gemini (with RAG)", gemini_results)
print_metrics("Gemini (without RAG)", gemini_no_rag_results)



===== GPT-2 (with RAG) =====
BLEU-2:              0.0270
F1:                  0.1056
Exact Match:         0.0200
Semantic Similarity: 0.3472

===== GPT-2 (without RAG) =====
BLEU-2:              0.0045
F1:                  0.0450
Exact Match:         0.0200
Semantic Similarity: 0.1953

===== Gemini (with RAG) =====
BLEU-2:              0.2554
F1:                  0.4221
Exact Match:         0.5600
Semantic Similarity: 0.5671

===== Gemini (without RAG) =====
BLEU-2:              0.0364
F1:                  0.1176
Exact Match:         0.4800
Semantic Similarity: 0.2560


### gird search
 Grid Search for Best Generation Parameters


In [ ]:
# #  Grid Search for Best Generation Parameters
# from itertools import product

# # Define parameter grid
# param_grid = {
#     "top_k": [1, 3, 5],
#     "max_new_tokens": [30, 50, 70],
#     "temperature": [0.7, 0.8, 1.0],
#     "top_p": [0.8, 0.9, 1.0]
# }

# # Generate all combinations
# param_combinations = list(product(
#     param_grid["top_k"],
#     param_grid["max_new_tokens"],
#     param_grid["temperature"],
#     param_grid["top_p"]
# ))

# best_params = None
# best_score = -1

# subset_df = df_val.head(20)

# for top_k, max_new_tokens, temperature, top_p in param_combinations:
#     predictions = []
#     references = []
#     similarities = []

#     for _, row in subset_df.iterrows():
#         question = row['question']
#         true_answer = row['answer_text']

#         generated_answer = generate_answer(
#             question,
#             top_k=top_k,
#             max_new_tokens=max_new_tokens
#         )

#         norm_pred = normalize_arabic(generated_answer)
#         norm_true = normalize_arabic(true_answer)

#         predictions.append(norm_pred)
#         references.append(norm_true)

#         emb_pred = similarity_model.encode(norm_pred, normalize_embeddings=True)
#         emb_ref = similarity_model.encode(norm_true, normalize_embeddings=True)
#         similarities.append(util.cos_sim(emb_pred, emb_ref).item())

#     # Evaluate: BLEU + F1 + Semantic Similarity (average as combined score)
#     smooth_fn = SmoothingFunction().method1
#     bleu_scores = [
#         sentence_bleu([r.split()], p.split(), smoothing_function=smooth_fn)
#         for r, p in zip(references, predictions)
#     ]
#     avg_bleu = np.mean(bleu_scores)

#     f1_scores = [f1_score(p, r) for p, r in zip(predictions, references)]
#     avg_f1 = np.mean(f1_scores)

#     avg_sim = np.mean(similarities)

#     combined_score = (avg_bleu + avg_f1 + avg_sim) / 3

#     if combined_score > best_score:
#         best_score = combined_score
#         best_params = {
#             "top_k": top_k,
#             "max_new_tokens": max_new_tokens,
#             "temperature": temperature,
#             "top_p": top_p
#         }

# print("Best Parameters:", best_params)
# print("Best Combined Score:", best_score)


Best Parameters: {'top_k': 3, 'max_new_tokens': 30, 'temperature': 0.7, 'top_p': 0.8}
Best Combined Score: 0.14345654434674376


# Chatbot interface

In [ ]:
import gradio as gr

def rag_pipeline(question, model_choice, use_rag):
    # Retrieve contexts only if RAG is enabled
    contexts = []
    if use_rag == "With RAG":
        contexts = retrieve_similar_context(question, top_k=1)

    # Pick model
    if model_choice == "GPT-2":
        if use_rag == "With RAG":
            answer = generate_answer_gpt2(question, top_k=1)
        else:
            answer = generate_answer_gpt2_no_rag(question)

    elif model_choice == "Gemini":
        if use_rag == "With RAG":
            answer = generate_answer_gemini(question, top_k=1)
        else:
            answer = generate_answer_gemini_no_rag(question)

    # Format retrieved contexts (only if RAG)
    retrieved_text = ""
    if use_rag == "With RAG":
        retrieved_text = "\n\n--- Retrieved Contexts ---\n"
        for c in contexts:
            retrieved_text += f"- {c['context'][:400]}...\n"

    return answer, retrieved_text


# Gradio UI Layout #

with gr.Blocks(theme="soft") as iface:
    gr.Markdown(
        """
        # **Chatbot**
        *Choose your model, enable RAG, ask your question — and watch the magic happen!*
        """
    )

    with gr.Row():
        model_choice = gr.Radio(
            ["GPT-2", "Gemini"],
            label="Choose Model",
            value="GPT-2"
        )

        use_rag = gr.Radio(
            ["With RAG", "Without RAG"],
            label="Use Retrieval (RAG)?",
            value="With RAG"
        )

    question = gr.Textbox(
        label=" Your Question",
        placeholder="اكتب سؤالك هنا..."
    )

    submit_btn = gr.Button(" Send")

    with gr.Row():
        answer_box = gr.Textbox(
            label=" Model Answer",
            lines=5
        )
        context_box = gr.Textbox(
            label=" Retrieved Context (If RAG)",
            lines=5
        )

    submit_btn.click(
        rag_pipeline,
        inputs=[question, model_choice, use_rag],
        outputs=[answer_box, context_box]
    )

iface.launch()


/tmp/ipython-input-4074520336.py:34: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme="soft") as iface:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1f052558c93b4f899f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
